In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = Chroma('500100',embedding_function = embedding, persist_directory='DataBase/db_500_100')

/usr/lib/python3.10/importlib/__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
/tmp/ipykernel_27797/2831752988.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langch

In [9]:
ids = vector_store.get()['ids']

In [42]:
import random
ids_quest = random.sample(ids, 10000)

In [18]:
# what in id
some_id = vector_store.get(ids = [ids_quest[1]])

In [19]:
some_id

{'ids': ['355aae89-c1c1-45e2-9fd5-d20ce6c0e11c'],
 'embeddings': None,
 'metadatas': [{'page': 2, 'source': './Data/1792125902228222608.pdf'}],
 'documents': ['ance InternationaL Evaluation Index (AGILE Index).  The index is utilized to delve into the sta-\ntus of AI governance to date in 14 countries for the first batch of evaluation.  The aim is to depict the \ncurrent state of AI governance in these countries through data scoring, assist them in identifying their \ngovernance stage  and uncover ing governance issues, and ultimately offer insights for the enhance-\nment of their AI governance systems.'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [33]:
import pandas as pd

questions_df = pd.DataFrame(columns = ['id', 'source', 'page', 'context', 'question', 'answer'])

In [24]:
import client
client.client('How are you?')

"I'm functioning properly. What would you like to know or discuss?"

In [43]:
for i, id in enumerate(ids_quest):
    id_doc = vector_store.get(ids = [id])
    context = id_doc['documents'][0]
    prompt = f"""
    Придумай 1 вопрос и кратко ответь на него по приложенному тексту. Вопрос должен быть таким, что ответить на него можно только прочитав данный текст. К каждому вопросу напиши краткий ответ.\
    Приложенный текст: {context}
    Формат вывода:\
    Вопрос: ...\
    Ответ: ...\
    """

    s = client.client(prompt)
    qu = s[:s.find('\n')].replace('Вопрос: ', '')
    an = s[s.find('\n'):].replace('\nОтвет: ', '')
    questions_df.loc[i] = [id, id_doc['metadatas'][0]['source'], id_doc['metadatas'][0]['page'], context, qu, an]
    #print(client.client(prompt))
    #print('Контекст:' , context)

In [62]:
import csv
for i in range(100):
    start = 100 * i
    stop = 100 * (i+1)
    name = 'quest_data/quest_answer_context_' + str(i) + '.csv'
    new_df = questions_df[start :stop]
    

In [64]:
try:
    questions_df.to_csv('quest_data/quest_answer_context_full' + '.csv')
except:
    pass

In [44]:
len(questions_df)

10000

In [61]:
questions_df[100:200].to_csv('1.csv')